# Mission to Mars: A Webscraping Commission

In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup as bs
from flask import Flask, render_template, redirect
# from flask_pymongo import PyMongo
from splinter import Browser
import requests
import time


#need to install library that has scrape_info, but I don't know what it is...!!!!
#import scrape_info

In [2]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
#browser.quit()

# Scrape News Headlines from NASA

In [5]:
# Scrape content from Nasa's Mars news site
url = "https://mars.nasa.gov/news/"

In [6]:
browser.visit(url)

In [7]:
html = browser.html

In [8]:
soup1 = bs(html, 'html.parser')

In [9]:
news_update = soup1.find('div', class_ = 'list_text')
news_update

<div class="list_text"><div class="list_date">September 13, 2018</div><div class="content_title"><a href="/news/8367/marco-makes-space-for-small-explorers/" target="_self">MarCO Makes Space for Small Explorers</a></div><div class="article_teaser_body">A pair of NASA CubeSats flying to Mars are opening a new frontier for small spacecraft.</div></div>

In [10]:
news_date = news_update.find('div', class_="list_date").text

In [11]:
news_headline = news_update.find('div', class_="content_title").text

In [12]:
news_teaser = news_update.find('div', class_="article_teaser_body").text

In [13]:
MarsUpdate={'news_date': news_date,
           'news_headline': news_headline,
           'news_teaser': news_teaser,
           }


In [14]:
print(news_date)
print(news_headline)
print(news_teaser)

September 13, 2018
MarCO Makes Space for Small Explorers
A pair of NASA CubeSats flying to Mars are opening a new frontier for small spacecraft.


# Scrape Images from JPL

In [15]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [16]:
html = browser.html
    
soup = bs(html, 'html.parser')

In [17]:
img = soup.find('div', class_='carousel_items')
img = img.findChild()
img = img.findAll()
img = img[0]
img = img.find('a', class_="button fancybox")
img = img.attrs['data-fancybox-href']

MarsUpdate['featureImage'] = img

#{'feat':img}

In [18]:
MarsUpdate

{'news_date': 'September 13, 2018',
 'news_headline': 'MarCO Makes Space for Small Explorers',
 'news_teaser': 'A pair of NASA CubeSats flying to Mars are opening a new frontier for small spacecraft.',
 'featureImage': '/spaceimages/images/mediumsize/PIA19673_ip.jpg'}

# Scrape Weather from Twitter

In [41]:
url = "https://twitter.com/marswxreport?lang=en"

In [42]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)
browser.visit(url)

In [43]:
html_3 = browser.html
soup_3 = bs(html, 'html.parser')

In [38]:
#find the div BEFORE you find the paragraph class!!!!! + .get_text() as a splinter command
#save HTML as a multiline variable with triple quotes
weather = soup_3.find('div', class_ = 'js-tweet-text-container')
# weather = soup_3.find('p', class_ = 'TweetTextSize TweetTextSize--normal js-tweet-text tweet-text')

weather
#for weather in mars_weather:
    
    #print(weather.text)

In [44]:
MarsUpdate['mars_weather'] = weather

In [45]:
MarsUpdate

{'news_date': 'September 13, 2018',
 'news_headline': 'MarCO Makes Space for Small Explorers',
 'news_teaser': 'A pair of NASA CubeSats flying to Mars are opening a new frontier for small spacecraft.',
 'featureImage': '/spaceimages/images/mediumsize/PIA19673_ip.jpg',
 'mars_weather': None,
 'mars_facts':         Fact Descriptor                          Value
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
 'hemisphere_images': {'url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg',


In [26]:
#<p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" lang="en" data-aria-label-part="0">Sol 2168 (2018-09-11), high -13C/8F, low -69C/-92F, pressure at 8.99 hPa, daylight 05:41-17:57</p>

# Scrape Factoids from Space Facts

In [27]:
url = "https://space-facts.com/mars/"

In [28]:
fact_table = pd.read_html(url)
fact_table

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [29]:
type(fact_table)

list

In [30]:
mars_facts_df = fact_table[0]
mars_facts_df.columns = ["Fact Descriptor", "Value"]
mars_facts_df.head()

,Fact Descriptor,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"


In [31]:
html_mars_facts = mars_facts_df.to_html()
html_mars_facts

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Fact Descriptor</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millen

In [32]:
MarsUpdate['mars_facts'] = mars_facts_df

In [33]:
MarsUpdate

{'news_date': 'September 13, 2018',
 'news_headline': 'MarCO Makes Space for Small Explorers',
 'news_teaser': 'A pair of NASA CubeSats flying to Mars are opening a new frontier for small spacecraft.',
 'featureImage': '/spaceimages/images/mediumsize/PIA19673_ip.jpg',
 'mars_weather': None,
 'mars_facts':         Fact Descriptor                          Value
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers}

# Hemisphere Images from USGS

In [34]:
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

In [35]:
hemispheres = ['Cerberus Hemisphere Enhanced',
              'Schiaparelli Hemisphere Enhanced',
              'Syrtis Major Hemisphere Enhanced',
              'Valles Marineris Hemisphere Enhanced']
links = []

# [  { title: '', url: '' },  ]
for hemisphere in hemispheres:
   browser.visit(url)
   time.sleep(5)
   print(f'chrome driver visiting url {url}')
   browser.click_link_by_partial_text(hemisphere)
   time.sleep(5)
   highresMars_html = browser.html
   soup = bs(highresMars_html, "html.parser")
   time.sleep(5)
   image_url_hemisphere = soup.find('div', class_='downloads').a['href']
   image_dict = {}
   image_dict['url'] = image_url_hemisphere
   image_dict['title'] = hemisphere
   links.append(image_dict)

chrome driver visiting url https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars
chrome driver visiting url https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars
chrome driver visiting url https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars
chrome driver visiting url https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars


In [36]:
MarsUpdate['hemisphere_images'] = image_dict

In [37]:


#list of the different hemisphere
#empty list of links
#pull out of jupyternb, indent then Scrape()function then 